In [62]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'composite_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'cpu', 'random_seed': 4567, 'data_configs': {'data_class': 'tinybig.data.composite_function', 'data_parameters': {'name': 'elementary_function', 'train_batch_size': 100, 'test_batch_size': 100, 'equation_index': 3}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 2000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.003, 'weight_decay': 0.0001}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.999}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [2, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_param

In [63]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  cpu ; random_seed:  4567


In [64]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  2441


In [65]:
print('loading dataset...')
data_loader = data_obj.load()
print(data_loader['str_equation'])

loading dataset...
C.3,3,f,exp(x+y)+ln(x+y),2,x,0,1,y,0,1


In [66]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device,
                                     metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 10.326179218292236, Test Score: 10.32617924140686, Time Cost: 0.01395106315612793
Epoch: 100, Test Loss: 7.914766920293914e-05, Test Score: 7.914767148930346e-05, Time Cost: 0.013875007629394531
Epoch: 200, Test Loss: 3.236313677916769e-05, Test Score: 3.2363136768184975e-05, Time Cost: 0.012770891189575195
Epoch: 300, Test Loss: 2.7063438938057516e-05, Test Score: 2.7063440235107427e-05, Time Cost: 0.013006210327148438
Epoch: 400, Test Loss: 1.942684921232285e-05, Test Score: 1.942684902786057e-05, Time Cost: 0.012937068939208984
Epoch: 500, Test Loss: 2.3585923145219566e-06, Test Score: 2.358592369078816e-06, Time Cost: 0.012963294982910156
Epoch: 600, Test Loss: 3.3832200188044226e-06, Test Score: 3.3832200367497346e-06, Time Cost: 0.01303720474243164
Epoch: 700, Test Loss: 2.792987447719497e-06, Test Score: 2.7929874163661883e-06, Time Cost: 0.012177228927612305
Epoch: 800, Test Loss: 5.203032696954324e-06, Test Score: 5.2030327682816565e-06, 

In [67]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [68]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 3.792877656964011e-07


In [69]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
